In [1]:
#Import libraries
import cv2
import smtplib 
import pywhatkit as k
import boto3
import boto.ec2
import time
import numpy as np
from os import listdir
from os.path import isfile, join
import os
from datetime import date

import imghdr
from email.message import EmailMessage

In [2]:
# import pywhatkit as k
# import time

# Sample collection

In [3]:


# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/mydata/'+ 'MyPhoto' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(5) == 13 or count == 300: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# Model Training

In [4]:
# import cv2
# import numpy as np
# from os import listdir
# from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/mydata/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

richard_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
richard_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


In [5]:
## This function recalculate the time passed to the pywhatkit's function used to send the message 
def timeClear(timeInSec,secs):
    timeInSec=int(timeInSec)
    secs=int(secs)
    if secs >=30:
        timeInSec += 120
    else:
        timeInSec += 60   
    return time.localtime(timeInSec) ## conver time in date


## Our Phone Numbers
#phNo = ['+917509490428', '+918580801728', '+919039318060', '+917250731457', '+917703042310', '+918580459580' ]
phNo = [ '+917250731457' ]

sendEmAddrs=['rishavvidya2018@gmail.com',]

receivEmAddrs=['rishavvidya2018@@gmail.com',]

message = """From: From SENDER_NAME <rishavvidya2018@gmail.com>
                                        To: RECEIVER_NAME <rishavkumarroy9999@gmail.com>
                                        Subject: Face Unlock Detected
                                        This is a test message To unlock Your device.....
                            """

In [6]:
def finalToday():
    t = time.localtime()  ## Get Time in formatted form 
    now = time.strftime("%H:%M:%S", t)
    cToday = date.today() 
    finalToday = cToday.strftime("%B %d, %Y")
    finalToday = finalToday+' At '+now
    return finalToday

In [7]:
    

def trySendEMAil(sender, passwd, receiver, dateT):
    newMessage = EmailMessage()                         
    newMessage['Subject'] = "YOUR FACE HAS BEEN DETECTED" 
    newMessage['From'] = sender                
    newMessage['To'] = receiver                  
    newMessage.set_content("------------------------------------------------------------------"+
                            '\nThis has been sent to you because your face has been detected today '+dateT+""".\n
    Here an attachment of image captured\n\n------------------------------------------------------------------\n""") 
    with open('./faces/Detected.png', 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(sender, passwd)   
        print("Login successfully...")
        smtp.send_message(newMessage)
        smtp.quit()
        print ("Successfully sent email")
    return
    

In [8]:
def launchEC2():
    ec2 = boto3.resource('ec2')  #Establish connection to ec2 instance   
    instance = ec2.create_instances(
                    ImageId = 'ami-0ad704c126371a549',  # use the AMI-linux type image
                    MinCount = 1,
                    MaxCount = 1,
                    InstanceType = 't2.micro',
                    KeyName = 'key-for-rdp-os',  # try use the key pair available on aws acc
                    SubnetId = 'subnet-2762834c')
    #
    instance_id = instance[0].id
    print ("The instance id is: ", instance_id)  
    #instance[0].id)
    print()
    time.sleep(5)
    print("finish launch instance")
    #
    conn = boto.ec2.connect_to_region("ap-south-1")
    print(conn)
    #
    ###To stop instance with boto    
    #stop = conn.stop_instances(instance_ids= instance_id) #['i-0638b07266fd59322'])
    #print("The stopped instance is: ", stop)
    #
    ## create volume of 5Gb     
    vol = conn.create_volume(5, "ap-south-1a") #launch EBS volume of 5GB
    print("Launched Volume is: ", vol)
    print()
    #
    ##get The status of the volume available or not
    curr_vol = conn.get_all_volumes([vol.id])[0]   # get all volumes availables
    s = curr_vol.status          # get the status of volume launched
    print("Volume Status: ", s)
    print()
    #
    #Attach the volume created 
    #if curr_vol is  None:
    time.sleep(100)
    conn.attach_volume (vol.id, instance_id, "/dev/sdx")  #Attach a volume to my instance created above
    print("Volume successfully attached...")
    print("Task Completed !!!!")
    return

# Deployement in live system

In [ ]:
# import cv2
# import numpy as np
# import os


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    key=0
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = richard_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hi Mr. Rishav Kumar Roy", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
            #Get the Date and time of facial recognized
            detectionTime=finalToday()
            
            cv2.imshow('Face Recognition', image )
            # write last captured image, it will be our attachment image in the mail. 
            cv2.imwrite('./faces/Detected.png',image)
            
            time.sleep(3)
            #
                ######## Send me mail when face detected
            ### For that you have to import smtplib library and install it before with "pip install secure-smtplib"
            sender = "rishavvidya2018@gmail.com"             #Put ur mail id
            receiver = 'rishavkumarroy9999@gmail.com'       #Put the receiver mail id here #pas = getpass.getpass()
            passwd =  open('pwd.txt', 'r').read()   #read from my local-file the passwd to access my gmail

            for i in phNo:
                t = time.localtime()  ## Get Time in formatted form 
                now = time.time() ## Get Time in seconds 
                ##hours=t.tm_hour  ## Extract number of hours
                #mins=t.tm_min    ## Extract number of minutes
                secs=t.tm_sec   ## Extract number of seconds
                t_F=timeClear(now, secs) 
                hours_F=t_F.tm_hour
                mins_F=t_F.tm_min
                secs_F=t_F.tm_sec
                print("Sending whatsapp message....\n")
                k.sendwhatmsg(i, "⚠⚠⚠A face has been detected🙂🙂\nCheck Your mailbox📧📬 for more information\n", hours_F, mins_F,wait_time= 25,tab_close=True)
            try:
                #smtpObj = smtplib.SMTP('localhost')
                trySendEMAil(sender, passwd, receiver, detectionTime)
                break
                
            except SMTPException: 
                print ("Error: unable to send email")
                pass
        else:
            cv2.putText(image, "Sorry! who are you?", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            time.sleep(3)
            print(" Your are not Owner, so w'll start Launch an  ec2 instance......!!! ")
            #
            # Launch an ec2 instances to my aws account       
            launchEC2()
            #key = 13
            break
            

            
            

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

Sending whatsapp message....

In 54 seconds web.whatsapp.com will open and after 25 seconds message will be delivered


In [22]:
import boto


boto.ec2.connect_to_region("ap-south-1")

EC2Connection:ec2.ap-south-1.amazonaws.com

In [23]:
#             ec2 = boto3.resource('ec2')  #Establish connection to ec2 instance   
#             instance = ec2.create_instances(
#                     ImageId = 'ami-0ad704c126371a549',  # use the AMI-linux type image
#                     MinCount = 1,
#                     MaxCount = 1,
#                     InstanceType = 't2.micro',
#                     KeyName = 'key-for-rdp-os',  # try use the key pair available on aws acc
#                     SubnetId = 'subnet-2762834c')
#             #
#             instance_id = instance[0].id
#             print ("The instance id is: ", instance_id)  
#             #instance[0].id)
#             print()
#             time.sleep(5)
#             print("finish launch instance")
#             #
#             conn = boto.ec2.connect_to_region("ap-south-1")
#             print(conn)
#             #
#             ###To stop instance with boto    
#             #stop = conn.stop_instances(instance_ids= instance_id) #['i-0638b07266fd59322'])
#             #print("The stopped instance is: ", stop)
#             #
#             ## create volume of 5Gb     
#             vol = conn.create_volume(5, "ap-south-1a") #launch EBS volume of 5GB
#             print("Launched Volume is: ", vol)
#             print()
#             #
#             ##get The status of the volume available or not
#             curr_vol = conn.get_all_volumes([vol.id])[0]   # get all volumes availables
#             s = curr_vol.status          # get the status of volume launched
#             print("Volume Status: ", s)
#             print()
#             #
#             #Attach the volume created 
#             #if curr_vol is  None:
#             time.sleep(100)
#             conn.attach_volume (vol.id, instance_id, "/dev/sdx")  #Attach a volume to my instance created above
#             print("Volume successfully attached...")
#             print("Task Completed !!!!")

In [24]:
sender = "rishavvidya2018@gmail.com"             #Put ur mail id
receiver = 'rishavkumarroy9999@gmail.com'       #Put the receiver mail id here #pas = getpass.getpass()
passwd =  open('pwd.txt', 'r').read()
detectionTime=finalToday()

trySendEMAil(sender, passwd, receiver, detectionTime)

Login successfully...
Successfully sent email
